# Football Parsing Code

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))

from os.path import join
from fsUtils import mkSubDir, setFile, isFile, removeFile, isDir
from ioUtils import getFile, saveFile
from timeUtils import clock, elapsed
from fileUtils import getBaseFilename, getBasename, getDirname
from webUtils import getWebData, getHTML
from timeUtils import printDateTime, getDateTime, addMonths
from searchUtils import findExt
from time import sleep
from random import random
import sys
import re
from datetime import timedelta
from collections import Counter

from espngames import output, espn
from espngames import game, team, season
from espngames import historical

import datetime as dt
start = dt.datetime.now()

print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-04-14 19:46:03.284261


# Order To Download ESPN Data

In [2]:
hist = historical()
#hist.downloadTeamStandings()
#hist.parseAndDownloadTeamYearlyStandings()
#hist.parseTeamYearlyStandings()
#hist.downloadGameData()
#hist.parseGameData()

# Extract Game Data

# Penalty

In [3]:
from penalty import penalty

# Play Start

# Class Imports

# Play By Play Analysis

In [73]:
%load_ext autoreload
%autoreload

from summary import playclass, drivedetailclass, driveclass, playstartclass
from playYards import playyards
from playResult import playtextclass
from possession import possessionclass, mixedpossession, playpossessionclass
from footballPlays import playtype, footballplay, penaltyplay, puntplay, kickoffplay, fieldgoalplay
from footballPlays import patplay, returnplay, downsplay, tbdplay, noplay, safetyplay
from footballPlays import timeoutplay, endplay, beginplay, sackplay, rushingplay, passingplay, fumbleplay, touchdownplay

from debug import debugclass
from changePossession import possessionchangeclass
from teamStatistics import teamstatisticsclass
from gameStatistics import teamplayers, gameplayers



class playbyplay(espn, output):
    def __init__(self):
        self.name = "playbyplay"
        espn.__init__(self)
        output.__init__(self)
        
        self.hist = None
        self.ptc  = playtextclass()
        
    def setHistorical(self, hist):
        self.hist = hist
    
    
    def parsePlayStart(self, playStartText, playNo, numPlays, playData, debug=False):
        if len(playStartText) == 0:
            return None
        
        ps = None
        
        down = "(st|nd|rd|th)"
        num  = "(\d+)"
        word = "(\w+)"
        goal = "(GOAL|goal|Goal)"
        prep = "(on|at)"
        amp  = "(&|and)"
        
        ## Look for match of type [1st and 10 at CAL 19]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, num, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3]
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for match of type [1st and GOAL at CAL 1]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, goal, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[6] ## replace goal with distance to goal
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for very near match without side of field [1st and 10 at 50]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}".format(num, down, amp, goal, prep, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3] ## replace goal with distance to goal
            startY = groups[5]
            side   = None
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        return ps
        raise ValueError("Could not parse [{0}] and [{1}]".format(playStartText, playData))
        
        
        
    
    def parseClockAndQuarter(self, playData, debug=False):
        gameclock = None
        quarterNo = None
        
        ## Check for end of quarter/half/game
        if sum([x in playData for x in ["End of the ", "End Of The ", "end of the ", "end of ", "End of "]]) > 0:
            gameclock = timedelta(minutes=0, seconds=0) 
            if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                quarterNo = 1
            elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half", "half", "Half"]]) > 0:
                quarterNo = 2
            elif sum([x in playData for x in ["3rd Quarter", "3rd quarter"]]) > 0:
                quarterNo = 3
            elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                quarterNo = 4
            elif "OT" in playData:
                quarterNo = "OT"
            else:
                quarterNo = "UNKNOWN END QTR"
                #raise ValueError("Could not determine quarter from {0}".format(playData))

                
        if gameclock is None and quarterNo is None:
            ## Check for start of quarter/half/game
            if sum([x in playData for x in ["Start of the ", "Start Of The ", "start of the ", "start of "]]) > 0:
                gameclock = timedelta(minutes=15, seconds=0) 
                if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                    quarterNo = 1
                elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half"]]) > 0:
                    quarterNo = 2
                elif sum([x in playData for x in ["3rd Quarter", "3rd quarter", "2nd Half", "2nd half"]]) > 0:
                    quarterNo = 3
                elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                    quarterNo = 4
                elif "OT" in playData:
                    quarterNo = "OT"
                else:
                    quarterNo = "UNKNOWN START QTR"
                    #raise ValueError("Could not determine quarter from {0}".format(playData))

        
        if gameclock is None and quarterNo is None:

            ## Check for specific format (HH:MM - 1st)
            m = re.match(r"\((.*?)\)", playData)
            if m is None:
                raise ValueError("Could not determine time and quarter from {0}".format(playData))

            try:
                text = m.group(0)
            except:
                raise ValueError("Could not extract regex result from {0}".format(playData))

               
            ## Check for overtime
            try:
                result = text[1:-1]
                if "OT" in result:
                    gameclock = timedelta(minutes=0, seconds=0) 
                    quarterNo = "OT"
            except:
                pass
            

            if gameclock is None:
                try:
                    result = text[1:-1]
                    result = result.split(" - ")
                    gameclock,quarter = result
                except:
                    raise ValueError("Could not parse time data: {0}".format(text))


                try:
                    minutes,seconds = gameclock.split(":")
                    gameclock = timedelta(minutes=int(minutes), seconds=int(seconds)) 
                except:
                    raise ValueError("Could not create timedelta for {0}".format(gameclock))


                quarterNo = {"1st":1, "2nd": 2, "3rd": 3, "4th": 4}.get(quarter)
                if quarterNo is None:
                    print("Could not find quarter from [{0}] or [{1}]".format(quarter, text))
                    #raise ValueError("Could not find quarter from [{0}] or [{1}]".format(quarter, text))


                try:
                    playData = playData.replace(text, "").strip()
                except:
                    raise ValueError("Could not remove [{0}] from [{1}]".format(text, playData))
                
        
        return gameclock,quarterNo,playData

        
    def parsePossession(self, possession, fieldMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(possession, list):
            raise ValueError("Possession is not a list: {0}".format(possession))
            
        if len(possession) != 1:
            raise ValueError("Not one element in possession list: {0}".format(possession))
            
        teamID = possession[0]
        
        teamAbbrev = None
        try:
            teamAbbrev = fieldMap[teamID]
        except:
            raise ValueError("Could not find {0} in field map: {1}".format(teamID, fieldMap))
            
        return teamAbbrev
        
        
    def augmentPlayStartWithPossession(self, possession, startVals, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if isinstance(startVals, playstartclass):
            
            ## Check for a team having the ball on their own side of the field (common)
            try:
                startY = int(startVals.startY)
            except:
                raise ValueError("Starting line {0} is not an integer".format(startVals.startY))

            if possession == startVals.side:
                distToEndZone = 100 - startY
            else:
                distToEndZone = startY                
                
            startVals.setStartY(startY)
            startVals.setDistToEndZone(distToEndZone)

        return startVals
    

    def parseHeadline(self, headline, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        pltype = None
        
        if isinstance(headline, list):
            if len(headline) >= 1:
                ptype  = playtype(headline[0])
                pltype = ptype.getPlay()
                
                if pltype is None:
                    raise ValueError("Could not determine headline play type: {0}".format(headline[0]))
            else:
                print("Not one headline entry: {0}".format(headline))
        else:
            raise ValueError("Headline is not a list: {0}".format(headline))
            
        return pltype
    
    
    def parseScore(self, score, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(score, list):
            raise ValueError("Could not determine score type: {0}".format(score))
            
        if len(score) != 1:
            raise ValueError("Not one detail entry: {0}".format(score))
            
        scoredata = score[0]
        
        try:
            scoredata = int(scoredata)
        except:
            raise ValueError("Could not find an integer score for {0}".format(scoredata))
            
        return scoredata
    
    
    def parseDetail(self, detail, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(detail, list):
            raise ValueError("Could not determine detail play type: {0}".format(detail))
            
        if len(detail) != 1:
            raise ValueError("Not one detail entry: {0}".format(detail))
            
        detaildata = detail[0]
        
        
        yards = "(yards|yard|Yds|yds|Yd|yd)"
        plays = "(play|plays)"        
        num  = "([+-?]\d+|\d+)"
        
        totalplays = None
        totalyards = None
        totalclock = None

        m = re.search(r"{0}\s{1},\s{2}\s{3},\s{4}:{5}".format(num, plays, num, yards, num, num), detaildata)
        if m is not None:
            groups = m.groups()
            totalplays = int(groups[0])
            totalyards = int(groups[2])
            totalclock = timedelta(minutes=int(groups[4]), seconds=int(groups[5]))

            
        if totalplays is None and totalyards is None and totalclock is None:
            m = re.search(r"{0}\s{1},\s{2}\s{3}".format(num, plays, num, yards), detaildata)
            if m is not None:
                groups = m.groups()
                totalplays = int(groups[0])
                totalyards = int(groups[2])
                totalclock = timedelta(minutes=0, seconds=0)
                
            
        if totalplays is None and totalyards is None and totalclock is None:
            raise ValueError("Could not parse drive detail: {0}".format(detaildata))

        drivedetail = drivedetailclass(plays=totalplays, yards=totalyards, gametime=totalclock)
            
        return drivedetail
    
    
    
        
        
        
    
    ########################################################################################################
    ##
    ## Analyze Possession For Every Play
    ##
    ########################################################################################################
    def analyzePossession(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
            
        unknown = False

        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            prevStart  = None
            for ipl,drivePlay in enumerate(drivePlays):
                play       = drivePlay.play
                valid      = drivePlay.valid
                if valid is False:
                    continue
                possession = drivePlay.possession
                text       = possession.text
                status     = ""
                if possession.isUnknownStart():
                    unknown = True
                    status  = "<---- UNKNOWN"
                if possession.isPreviousStart():
                    if ipl == 0:
                        unknown = True
                        status  = "<---- UNKNOWN"
                    else:
                        try:
                            nextStart = drivePlays[ipl+1].possession.start
                        except:
                            nextStart = None
                        
                        if nextStart == prevStart:
                            possession.start = prevStart
                            status  = "<---- PREV/NXT"
                        else:
                            status  = "<---- PREV/UNK"
                            
                if isinstance(play, (rushingplay, passingplay)):
                    prevStart = possession.start
                keys = ",".join([k for k,v in play.pa.__dict__.items() if v is True])
                if status == "":
                    status = keys
                if debug:
                    print("{0}\t{1}\t{2}\t{3: <15}{4: <15}{5: <5}{6: <40}{7}".format(idr,ipl,str(possession.start),str(play.name),str(possession.player),str(possession.position),str(status),str(text)))
            if debug:
                print("")

        if unknown is True:
            if debug:
                print("UNKNOWN PLAY START IN GAME")
            
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Kickoff Structure
    ##
    ########################################################################################################
    def analyzeKickoffs(self, gameData, players, copMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
                        
        unknown = False
        
        kickers = {}
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            prevStart  = None
            if sum([isinstance(x.play, kickoffplay) for x in drivePlays]) > 0:
                posCntr = Counter()
                for playData in drivePlays:
                    if isinstance(playData.play, (rushingplay, passingplay, sackplay, fieldgoalplay)):
                        posCntr[playData.possession.start] += 1
                        
                if len(posCntr) == 0:
                    continue
                
                for ipl,playData in enumerate(drivePlays):
                    if isinstance(playData.play, (kickoffplay)):
                        playText = playData.play.text
                        keys = [" kickoff ", " Kickoff", " KICKOFF "]
                        pos = max([playText.find(x) for x in keys])
                        if pos > 0:
                            text = playText[:pos].strip()
                            if kickers.get(text) is None:
                                kickers[text] = Counter()
                            try:
                                team = copMap[posCntr.most_common(1)[0][0]]
                            except:
                                if debug:
                                    print("Unknown team: {0} in {1}".format(team, fname))
                                continue
                                
                            try:
                                kickers[text][team] += 1
                            except:
                                pass
                            
                        
        for kicker in kickers.keys():
            kickers[kicker] = kickers[kicker].most_common(1)[0][0]
            if debug:
                print("Kicker: {0} ---> {1}".format(kicker,kickers[kicker]))
        players.addKickers(kickers, debug)
        
        
        for idr,driveData in enumerate(gameData):
            for ipl,playData in enumerate(driveData.plays):
                if isinstance(playData.play, (kickoffplay)):
                    poss, self.determinePossession(playData.play, players)
                    gameData[idr].plays[ipl].possession = poss

        gameData = self.analyzePossession(gameData, players, debug)
            
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Returns Structure
    ##
    ########################################################################################################
    def analyzeReturns(self, gameData, players, copMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for ipl,playData in enumerate(drivePlays):
                valid      = playData.valid
                if valid is False:
                    continue
                if isinstance(playData.play, returnplay):
                    if debug:
                        print("Return play {0},{1} --> {2}: {3}".format(idr,ipl,playData.possession.start,playData.play.text))
                    if not playData.possession.isKnownStart():
                        prevPlay = drivePlays[ipl-1]
                        if prevPlay.possession.isKnownStart():
                            playData.possession.start = copMap[prevPlay.possession.start]
                            if debug:
                                print("\tReturn play {0},{1} --> {2}: {3}".format(idr,ipl,playData.possession.start,playData.play.text))
                    
                            for ipl2 in range(ipl+1, len(drivePlays)):
                                if not drivePlays[ipl2].possession.isKnownStart():
                                    drivePlays[ipl2].possession.start = drivePlays[ipl2-1].possession.start
                                    if debug:
                                        print("\tSetting {0},{1} start to {2}".format(idr, ipl2, playData.possession.start))

                    
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze PAT Structure
    ##
    ########################################################################################################
    def analyzePATs(self, gameData, players, copMap, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for ipl,playData in enumerate(drivePlays):
                valid      = playData.valid
                if valid is False:
                    continue
                if isinstance(playData.play, patplay):
                    if debug:
                        print("PAT play {0},{1} --> {2}: {3}".format(idr,ipl,playData.possession.start,playData.play.text))
                    if not playData.possession.isKnownStart():
                        if playData.play.pa.getKey("defpat"):
                            prevPlay = drivePlays[ipl-1]
                            if prevPlay.possession.isKnownStart():
                                playData.possession.start = copMap[prevPlay.possession.start]
                                if debug:
                                    print("\tPAT play {0},{1} --> {2}: {3}".format(idr,ipl,playData.possession.start,playData.play.text))
                    
                                for ipl2 in range(ipl+1, len(drivePlays)):
                                    if not drivePlays[ipl2].possession.isKnownStart():
                                        drivePlays[ipl2].possession.start = drivePlays[ipl2-1].possession.start
                                        if debug:
                                            print("\tSetting {0},{1} start to {2}".format(idr, ipl2, playData.possession.start))

                    
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Interception Structure
    ##
    ########################################################################################################
    def analyzeInterceptions(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for ipl,playData in enumerate(drivePlays):
                valid      = playData.valid
                if valid is False:
                    continue
                if playData.play.pa.getKey("interception") is True:                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass
                        
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Penalties and Timeouts
    ##
    ########################################################################################################
    def analyzePenaltiesAndTOs(self, gameData, players, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
        
        lastPlayPoss = None
        
        for idr,driveData in enumerate(gameData):
            if lastPlayPoss is not None:
                if len(driveData.plays) == 0:
                    if debug:
                        print("There are no plays in drive number {0}".format(idr))
                    continue
                firstPlayPoss = driveData.plays[0]
                if not firstPlayPoss.possession.isKnownStart():
                    if debug:
                        print("Setting possession for first play to {0}".format(lastPlayPoss.start))
                    driveData.plays[0].possession.start = lastPlayPoss.start
                
            for ipl,playData in enumerate(driveData.plays):
                if isinstance(playData.play, (penaltyplay,timeoutplay)):
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].possession.start == gameData[idr].plays[ipl+1].possession.start:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass
                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:

                        try:
                            if gameData[idr].plays[ipl+1].possession.isPreviousStart() is False and gameData[idr].plays[ipl+1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl+1].possession.start
                        except:
                            pass
                    
                    if playData.possession.isPreviousStart() is True or playData.possession.isUnknownStart() is True:
                        try:
                            if gameData[idr].plays[ipl-1].valid is True:
                                gameData[idr].plays[ipl].possession.start = gameData[idr].plays[ipl-1].possession.start
                        except:
                            pass

            lastPlayPoss = driveData.plays[-1].possession



        gameData = self.analyzePossession(gameData, players, debug)
                        
        return gameData
        
    
    ########################################################################################################
    ##
    ## Analyze Interception Structure
    ##
    ########################################################################################################
    def analyzeEndOfGame(self, gameData, players, postDriveScores, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
             
        homeTeam = players.homeTeamName
        awayTeam = players.awayTeamName

        runningHomeScore = 0
        runningAwayScore = 0
        
        dc  = debugclass()

        postDriveScores["Analysis"] = []
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for drivePlay in drivePlays:
                if drivePlay.valid is False:
                    continue
                play       = drivePlay.play
                possession = drivePlay.possession
                score      = play.pa.isScore()
                if score > 0:
                    if possession.start == homeTeam:
                        runningHomeScore += score
                    elif possession.start == awayTeam:
                        runningAwayScore += score
                    else:
                        print("Possession {0} is neither home {1} or away {2}".format(possession.start, homeTeam, awayTeam))
                                    
            postDriveScores["Analysis"].append([runningAwayScore, runningHomeScore])
    
        ## Get penultimate drive
        penultKnownDrive = postDriveScores["Drives"][-2]
        penultAnalyDrive = postDriveScores["Analysis"][-2]
        if penultKnownDrive == penultAnalyDrive:
            lastKnownDrive = postDriveScores["Drives"][-1]
            lastAnalyDrive = postDriveScores["Analysis"][-1]
            finalScore = postDriveScores["Final"]
            if lastKnownDrive == lastAnalyDrive and lastKnownDrive == finalScore:
                if debug:
                    print("SCORE is CORRECT")
            else:
                if finalScore[0] - lastAnalyDrive[0] == 6:
                    ## Away Team                    
                    if debug:
                        print("  Away team ({0}) needs a TD".format(awayTeam))
                        dc.showDrive(gameData[-1], len(gameData), debug=debug)
                    for i in range(1,len(gameData[-1].plays)+1):
                        if gameData[-1].plays[-1*i].valid is False:
                            continue
                        lastPoss = gameData[-1].plays[-1*i].possession.start
                        print("  Last Possession goes to {0}".format(lastPoss))

                        if lastPoss == awayTeam:
                            gameData[-1].plays[-1*i].play.pa.touchdown = True
                        else:
                            print("    Problem with possession. Not sure....")
                        break
                elif finalScore[1] - lastAnalyDrive[1] == 6:
                    ## Home Team                
                    if debug:
                        print("  Home team ({0}) needs a TD".format(homeTeam))
                        dc.showDrive(gameData[-1], len(gameData), debug=debug)
                    for i in range(1,len(gameData[-1].plays)+1):
                        if gameData[-1].plays[-1*i].valid is False:
                            continue
                        lastPoss = gameData[-1].plays[-1*i].possession.start
                        print("  Last Possession goes to {0}".format(lastPoss))

                        if lastPoss == homeTeam:
                            gameData[-1].plays[-1*i].play.pa.touchdown = True
                        else:
                            print("    Problem with possession. Not sure....")
                        break                        
                else:
                    if debug:
                        print("Problems with the scores...")
                        print("KNOWN: {0}".format(lastKnownDrive))
                        print("ANALY: {0}".format(lastAnalyDrive))
                        print("FINAL: {0}".format(finalScore))
        else:
            if debug:
                scores = zip(postDriveScores["Drives"], postDriveScores["Analysis"])
                for i,score in enumerate(scores):
                    print(i,'\t',score)
            
        return gameData
    
    
    
    ########################################################################################################
    ##
    ## Analyze Game Score After Each Drive
    ##
    ########################################################################################################
    def analyzeGameScore(self, gameData, players, homeTeamGameData, awayTeamGameData, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        if not isinstance(gameData, list):
            raise ValueError("Game data is {0}".format(type(gameData)))

        homeTeam = players.homeTeamName
        awayTeam = players.awayTeamName

        finalHomeScore = None
        runningHomeScore = 0
        finalAwayScore = None
        runningAwayScore = 0
        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}\t(Post Drive Score)".format("Drive", "H", "(H)", "A", "(A)"))
            
        for idr in range(len(gameData)):
            driveData  = gameData[idr]
            homescore  = driveData.postdrivehomescore
            awayscore  = driveData.postdriveawayscore
            finalAwayScore = awayscore
            finalHomeScore = homescore

            drivePlays = driveData.plays
            if isinstance(drivePlays, list) is False:
                raise ValueError("Drive Plays is not a list object!")

            scoringplays = []
            for drivePlay in drivePlays:
                if drivePlay.valid is False:
                    continue
                play       = drivePlay.play
                possession = drivePlay.possession
                score      = play.pa.isScore()
                if score > 0:
                    scoringplays.append(play.text)
                    if possession.start == homeTeam:
                        runningHomeScore += score
                    elif possession.start == awayTeam:
                        runningAwayScore += score
                    else:
                        print("Possession {0} is neither home {1} or away {2}".format(possession.start, homeTeam, awayTeam))
                        
            if debug:
                print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format(idr, runningHomeScore, homescore, runningAwayScore, awayscore, scoringplays))
                
                
        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("----", "---", "---", "---", "---", ""))
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("END", runningHomeScore, finalHomeScore, runningAwayScore, finalAwayScore, ""))
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("TRUE", homeTeamGameData.teamAScore, "", awayTeamGameData.teamAScore, "", ""))
        
        diffHomeScore = homeTeamGameData.teamAScore - runningHomeScore
        diffAwayScore = awayTeamGameData.teamAScore - runningAwayScore

        if debug:
            print("{0: <7}{1: <4}{2: <4}{3: <4}{4: <4}{5}".format("DIFF", diffHomeScore, "", diffAwayScore, "", ""))
            
        if diffHomeScore != 0 or diffAwayScore != 0:
            return False
        return True
    
    
                        
    ######################################################
    ## Save Augmented Stats Data
    ######################################################
    def saveAugmentedStatsData(self, players, homeTeamMetaData, awayTeamMetaData, debug=False):
        if debug:
            fname = sys._getframe().f_code.co_name
            print("FUNC {0}".format(fname))
            
        augmentedStatsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats-extra.json".format(year))
        if isFile(augmentedStatsFilename) and False:
            augmentedStatsData = getFile(augmentedStatsFilename)
        else:
            augmentedStatsData = {}

        if augmentedStatsData.get(homeTeamMetaData["ID"]) is None:
            augmentedStatsData[homeTeamMetaData["ID"]] = {}
        if augmentedStatsData[homeTeamMetaData["ID"]].get('Kickers') is None:
            augmentedStatsData[homeTeamMetaData["ID"]]['Kickers'] = {}
        if augmentedStatsData[homeTeamMetaData["ID"]].get('Punters') is None:
            augmentedStatsData[homeTeamMetaData["ID"]]['Punters'] = {}

        for name in players.homeTeamPlayers.kickers:
            augmentedStatsData[homeTeamMetaData["ID"]]['Kickers'][name] = "K"
        for name in players.homeTeamPlayers.punters:
            augmentedStatsData[homeTeamMetaData["ID"]]['Punters'][name] = "P"


        if augmentedStatsData.get(awayTeamMetaData["ID"]) is None:
            augmentedStatsData[awayTeamMetaData["ID"]] = {}
        if augmentedStatsData[awayTeamMetaData["ID"]].get('Kickers') is None:
            augmentedStatsData[awayTeamMetaData["ID"]]['Kickers'] = {}
        if augmentedStatsData[awayTeamMetaData["ID"]].get('Punters') is None:
            augmentedStatsData[awayTeamMetaData["ID"]]['Punters'] = {}

        for name in players.awayTeamPlayers.kickers:
            augmentedStatsData[awayTeamMetaData["ID"]]['Kickers'][name] = "K"
        for name in players.awayTeamPlayers.punters:
            augmentedStatsData[awayTeamMetaData["ID"]]['Punters'][name] = "P"

        if debug:
            print("Saving augmented stats to {0}".format(augmentedStatsFilename))
        saveFile(idata=augmentedStatsData, ifile=augmentedStatsFilename, debug=True)                        
        
        
    
    
    
    
    
    
    ########################################################################################################
    ##
    ## Combine all play data data together
    ##
    ########################################################################################################
    def parsePlayData(self, playStart, playData, fieldMap, possData, gameID, driveNo, playNo, debug=False, verydebug=False):
        if verydebug:
            fname = sys._getframe().f_code.co_name
            print("\n  FUNC {0}({1},{2}: {3})".format(fname, driveNo, playNo, playData))
        
        
        ##########################
        ## Find clock and quarter
        ##########################
        gameclock, quarter, playText = self.parseClockAndQuarter(playData, debug=verydebug)

        
        ### Must fix corrupt text from time to time...
        playText,keep = self.editText(playText, gameID, driveNo, playNo)
        if keep is False:
            return None
        
        
        ##########################
        ## Determine play type
        ##########################
        ptype = playtype(playText)
        play  = ptype.getPlay()
        if ptype.valid is True:
            if play is None:
                raise ValueError("Could not determine play type from {0}".format(playText))
            play.analyze(debug=debug)
        if debug:
            if play is not None:
                print("\tPlay: {0} \t{1}".format(play.name, play.text))

        
        ##########################
        ## Check for valid play
        ##########################
        if ptype.valid is True and play is not None:
            validPlay = play.pa.getKey("noplay") != True
            if isinstance(play, (beginplay, endplay, timeoutplay)):
                validPlay = False
        else:
            validPlay = False
        
        
        
        ##########################
        ## Determine possession
        ##########################        
        if ptype.valid is True and play is not None and validPlay is True:
            playPossession = possData.determinePossession(play, debug=debug, verydebug=verydebug)
            if playPossession.start is None:
                raise ValueError("Possession for play of type [{0}] with text [{1}] was set to NONE!".format(play.name,playText))
        else:
            playPossession = playpossessionclass(start=None, end=None)
            playPossession.valid = False
        
        if playPossession.isForced() is not None:
            play.pa.forced = playPossession.isForced()            
        
        if playPossession.valid is False:
            validPlay = False
        
        if playPossession.valid is True:
            if playPossession.start is None:
                raise ValueError("Possession for play of type [{0}] with text [{1}] was set to NONE!".format(play.name,playText))
            
        
        ##########################
        ## Insert Clock/Quarter
        ##########################
        if playStart is None:
            playStart = playstartclass()
            
        if isinstance(playStart, playstartclass):
            playStart.setQuarter(quarter)
            playStart.setGameClock(gameclock)

            
        ### Result of play
        playResult = playclass(possession=playPossession, start=playStart, play=play, valid=validPlay)
        return playResult
    
    
    
    ################################################################################################################################################
    ##
    ##
    ## Main Loop To Process Games
    ##
    ##
    ################################################################################################################################################
    def parseGames(self, gameID=None, test=False, debug=False, verydebug=False):
        if self.hist is None:
            raise ValueError("Must set historical class!")
            
        if verydebug:
            debug = True

        self.unknownPlays = []
        
        self.toughParsing  = ['400547724']
        self.poorlyParsed  = ['401012731', '400547781', '400547808', '400548070', '400548428', '400610207', '400547822']
        self.poorlyParsed += ['400547970', '400547835', '400548026', '400548167', '400547827', '400548458']
        self.poorlyParsed += ['400547901', '400547976', '400548246', '400548278', '400548292', '400548448']


        
        self.statsToGet = {}
        self.badGames   = {}
        self.goodGames  = {}
        
        pcc = possessionchangeclass()
        dc  = debugclass()
            
        files    = findExt(self.hist.getGamesResultsDir(), ext=".p", debug=debug)
        for ifile in files:
            print(ifile)
            try:
                year = int(getBaseFilename(ifile).split("-")[0])
            except:
                raise ValueError("Could not get year from {0}".format(ifile))
            
            if year != 2018:
                continue
                
            yearData = getFile(ifile)
            
            seasonFilename = setFile(self.hist.getSeasonResultsDir(), "{0}.p".format(year))
            seasonData     = getFile(seasonFilename)
            
            statsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats.json".format(year))
            statsData     = getFile(statsFilename)
            
            augmentedStatsFilename = setFile(self.hist.getStatisticsResultsDir(), "{0}-stats-extra.json".format(year))
            augmentedStatsData     = getFile(augmentedStatsFilename)
                

            if len(self.unknownPlays) > 75:
                break


            
            
            totalGames = 0
            for gameIdent,gameData in yearData.items():
                if gameID is not None:
                    if gameID != gameIdent:
                        continue
                
                teamsMetaData    = gameData["Teams"]
                homeTeamMetaData = teamsMetaData["Home"]
                awayTeamMetaData = teamsMetaData["Away"]
                driveData        = gameData["Plays"]
                
                
                print("GAME ID: {0}".format(gameIdent))

                fieldMap = {}
                fieldMap[homeTeamMetaData["ID"]]     = homeTeamMetaData["Abbrev"]
                fieldMap[homeTeamMetaData["Abbrev"]] = homeTeamMetaData["ID"]
                fieldMap[awayTeamMetaData["ID"]]     = awayTeamMetaData["Abbrev"]
                fieldMap[awayTeamMetaData["Abbrev"]] = awayTeamMetaData["ID"]
                
                fieldMap["Home"] = homeTeamMetaData["Abbrev"]
                fieldMap["Away"] = awayTeamMetaData["Abbrev"]
                
                copmap = {}
                copmap[homeTeamMetaData["Abbrev"]] = awayTeamMetaData["Abbrev"]
                copmap[awayTeamMetaData["Abbrev"]] = homeTeamMetaData["Abbrev"]

                
                homeTeamSeasonData = seasonData.teams.get(homeTeamMetaData["ID"])
                if homeTeamSeasonData is None:
                    continue
                    raise ValueError("There is no home team season data for {0}, {1}".format(homeTeamMetaData["Abbrev"], homeTeamMetaData["ID"]))
                homeTeamGameData = [x["Game"] for x in homeTeamSeasonData.games if x["Game"].gameID == gameIdent]
                try:
                    homeTeamGameData = homeTeamGameData[0]
                except:
                    continue
                    raise ValueError("There is no home team game data")

                awayTeamSeasonData = seasonData.teams.get(awayTeamMetaData["ID"])
                if awayTeamSeasonData is None:
                    continue
                    raise ValueError("There is no away team season data for {0}, {1}".format(awayTeamMetaData["Abbrev"], awayTeamMetaData["ID"]))
                awayTeamGameData = [x["Game"] for x in awayTeamSeasonData.games if x["Game"].gameID == gameIdent]
                try:
                    awayTeamGameData = awayTeamGameData[0]
                except:
                    continue
                    raise ValueError("There is no away team game data")

                
                


                if gameIdent in self.hist.noGameData:
                    continue
                    
                if gameIdent in self.poorlyParsed:
                    continue
                    
                if gameIdent in self.toughParsing:
                    continue
                                
                
                ################################################################################
                ### Learn key plays for use in determining possession
                ################################################################################
                players = gameplayers(teamsMap=fieldMap, statsData=statsData)
                players.augmentData(augmentedStatsData)
                
                possData = possessionclass(players)
                
                if debug:
                    print("")
                    print("  Home Team: {0}".format(homeTeamMetaData))
                    print("\tPassers\t",players.homeTeamPlayers.passers)
                    print("\tRunners\t",players.homeTeamPlayers.runners)
                    print("\tKickers\t",players.homeTeamPlayers.kickers)
                    print("\tFGKickers\t",players.homeTeamPlayers.fgkickers)
                    print("\tPunters\t",players.homeTeamPlayers.punters)
                    print("")
                    print("  Away Team: {0}".format(awayTeamMetaData))
                    print("\tPassers\t",players.awayTeamPlayers.passers)
                    print("\tRunners\t",players.awayTeamPlayers.runners)
                    print("\tKickers\t",players.awayTeamPlayers.kickers)
                    print("\tFGKickers\t",players.awayTeamPlayers.fgkickers)
                    print("\tPunters\t",players.awayTeamPlayers.punters)
                    print("")


                
                if test:
                    saveFile(idata=driveData, ifile="testDriveData.p", debug=True)
                    saveFile(idata=fieldMap,  ifile="testFieldMap.p",  debug=True)
                    saveFile(idata=statsData,  ifile="testGameStats.p", debug=True)
                    saveFile(idata=augmentedStatsData,  ifile="testAugStats.p", debug=True)
                    1/0

                if len(self.unknownPlays) > 75:
                    break
                    
                
                gameResult = []
                totalPlays = 0
                
                
                ################################################################################
                ### Collect Post Drive Scores
                ################################################################################
                postDriveScores = {"Drives": [], "Final": []}
                postDriveScores["Final"] = [awayTeamGameData.teamAScore, homeTeamGameData.teamAScore]
                
                
                ################################################################################
                ### Iterate over drives
                ################################################################################
                for idr,drive in enumerate(driveData):
                    ### Fix known drive problems
                    #drive = self.fixDrive(gameIdent, idr, drive)
                    
                    driveNo    = drive['Drive']
                    headline   = drive['Headline']
                    detail     = drive['Detail']
                    ### For whatever reason home/away scores are reversed on the webpage...
                    homescore  = drive['AwayScore']
                    awayscore  = drive['HomeScore']
                    possession = drive['Posession'] ## note this needs to change later
                    drivedata  = drive['Data']
                    
                    
                    
                    if debug:
                        print("Drive: {0} \t{1} (H) - {2} (A)".format(idr,homescore, awayscore))
                    
    
                    ## Determine possession as defined in the play start fields later
                    possession = self.parsePossession(possession, fieldMap)
                    headline   = self.parseHeadline(headline)
                    detail     = self.parseDetail(detail)
                    homescore  = self.parseScore(homescore)
                    awayscore  = self.parseScore(awayscore)
                    
                    postDriveScores["Drives"].append([awayscore, homescore])
            
        
                    fullDriveData = driveclass(headline=headline, detail=detail, possession=possession,
                                              postdrivehomescore=homescore, postdriveawayscore=awayscore)
                    
                    drivePlays = []
            
                
                    ################################################################################
                    ### Iterate over plays in drive
                    ################################################################################
                    for ipl,play in enumerate(drivedata):
                        playNo        = play['Play']
                        playStartText = play['Start']
                        playData      = play['Data']
                        

                        ### Determine play starting position and clock
                        startVals = self.parsePlayStart(playStartText, playNo, len(drivedata), playData, debug=verydebug)
                        startVals = self.augmentPlayStartWithPossession(possession, startVals, debug=False)
                        
                        ### Determine type of play
                        playResult = self.parsePlayData(startVals, playData, fieldMap, possData, gameIdent, idr, ipl, debug=False, verydebug=verydebug)
                        if playResult is None:
                            continue
                        
                        if playResult.possession.start is None and playResult.valid is True:
                            raise ValueError("ERROR WITH POSSESSION:",playResult.play.name,'\t',playResult.play.text)
                        
                        totalPlays += 1
                        drivePlays.append(playResult)
                        
                        ### Check if we need to insert a play
                        newPlay = self.addPlay(gameIdent, idr, ipl, playResult, possData, verydebug)
                        if newPlay is not None:
                            totalPlays += 1
                            drivePlays.append(newPlay)
                        


                    
                    fullDriveData.setPlays(drivePlays)
                    #fullDriveData = self.insertMissingData(gameIdent, idr, fullDriveData, debug)
                    #fullDriveData = self.augmentPlayWithScore(fullDriveData, fieldMap, debug)
                    gameResult.append(fullDriveData)


                
                
                ################################################################################
                ### Analyze Possession
                ################################################################################ 
                dc.showGame(gameResult, debug=debug)
                gameResult = self.analyzePossession(gameResult, players, debug=debug)
                #gameResult = self.analyzeKickoffs(gameResult, players, copmap, debug=debug)
                #gameResult = self.analyzeInterceptions(gameResult, players, debug=debug)
                gameResult = pcc.splitChangeOfPossession(gameResult, players, copmap, debug=debug)
                gameResult = self.analyzePenaltiesAndTOs(gameResult, players, debug=debug)
                gameResult = self.analyzePossession(gameResult, players, debug=debug)
                gameResult = self.analyzeReturns(gameResult, players, copmap, debug=debug)
                gameResult = self.analyzePATs(gameResult, players, copmap, debug=debug)
                gameResult = self.analyzeEndOfGame(gameResult, players, postDriveScores, debug=debug)
                
                dc.showGame(gameResult, debug=debug)
                
                scoreResult = self.analyzeGameScore(gameResult, players, homeTeamGameData, awayTeamGameData, debug=True)
                if scoreResult is False:
                    if gameID is None:
                        self.badGames[gameIdent] = True
                        continue
                    dc.showGame(gameResult, debug=True)
                    
                    saveFile(idata=driveData, ifile="testDriveData.p", debug=True)
                    saveFile(idata=fieldMap,  ifile="testFieldMap.p",  debug=True)
                    saveFile(idata=statsData,  ifile="testGameStats.p", debug=True)
                    saveFile(idata=augmentedStatsData,  ifile="testAugStats.p", debug=True)
                    
                    raise ValueError("There was a problem parsing the scores for this game {0}!".format(gameIdent))
                else:
                    self.goodGames[gameIdent] = True
                #self.saveAugmentedStatsData(players, homeTeamMetaData, awayTeamMetaData, debug)
                
                
                #self.analyzeGameData(gameResult)


                    
                totalGames += 1
                if debug:
                    print("Found {0} plays in this game {1}".format(totalPlays, gameID))
                    
            print("Found {0} total games for {1}".format(totalGames, ifile))


    ########################################################################################################
    ## Necessary Edits To The Text
    ########################################################################################################
        
        
    def editText(self, text, gameID, driveNo, playNo):
        newtext = text
        keep    = True
        if gameID == "400547673":
            if driveNo == 15 and playNo == 14:
                newtext = "Lazedrick Thompson run for 4 yds for a TD, (Andrew DiRocco MISSED)"
        if gameID == "400547680":
            if driveNo == 29 and playNo == 3:
                newtext = "(3OT) Terrence Franks 2 Yd Run for a TD"
        if gameID == "400547693":
            if driveNo == 9 and playNo == 2:
                newtext = "Garrett Krstich sacked by Jeff Luc for a loss of 6 yards Garrett Krstich fumbled, recovered by Cincy Nick Temple , return for 0 yards , return for 27 yds (Two-Point Pass Conversion Failed)"
            if driveNo == 9 and playNo > 2:
                keep = False
        if gameID == '400548197':
            if driveNo == 6 and playNo == 12-1:
                keep = False
        if gameID == '400547890':
            #print(gameID,driveNo,playNo,'\t',text)
            if driveNo == 21 and playNo == 9-1:
                newtext = "Dylan Cantrell 39 Yd pass from Patrick Mahomes for a TD"
        if gameID == '400548388':
            if driveNo == 31 and playNo == 3-1:
                newtext = "(3OT) Matt Jones 1 Yd Run for a TD"
        if gameID == '400548061':
            if driveNo == 1 and playNo == 2-1:
                keep = False
        if gameID == "400548315":
            if driveNo == 15 and playNo == 0:
                keep = False
        if gameID == "400547757":
            if driveNo == 21 and playNo == 3:
                newtext = "(2OT) Thomas Sirk 5 Yd Run for a TD"
        if gameID == '400548151':
            if driveNo == 0 and playNo == 2:
                newtext = "Joe Licata pass intercepted for a TD DeAndre Scott return for 37 yds for a TD (PAT)"
        if gameID == '400547710':
            if driveNo == 30 and playNo == 1:
                newtext = "Terrell Hartsfield 27 Yd Fumble Return for a TD"                
        if gameID == '400547816':
            if driveNo == 14 and playNo == 11:
                keep = False     
        if gameID == '400548301':
            if driveNo == 6 and playNo == 2:
                newtext = "Sefo Liufau sacked by Anthony Lopez for a loss of 7 yards Sefo Liufau fumbled, recovered by Ariz Tra'Mayne Bondurant , return for 0 yards , return for 22 yds for a TD (PAT)"                
        if gameID == '400609076':
            if driveNo == 30 and playNo == 0:
                newtext = "Jerrard Randall 25 Yd Run for a TD"  
        if gameID == '400548444':
            if driveNo == 15 and playNo == 7:
                keep = True
                newtext = "Teldrick Morgan 42 Yd pass from Tyler Rogers for a TD (Defensive PAT)"
        if gameID == '400548000':
            if driveNo == 11 and playNo == 4:
                newtext = "Melvin Gordon run for 14 yds for a TD (PAT)"   
            if driveNo == 23 and playNo == 7:
                newtext = "Anthony Jennings pass complete to John Diarse for 35 yds for a TD (Two pt pass, Anthony Jennings pass to Trey Quinn GOOD)"                
        if gameID == '400547966':
            if driveNo == 3 and playNo == 0:
                newtext = "Ryan Santoso kickoff for 61 yds , De'Mornay Pierson-El return for 16 yds to the Neb 20."    
        if gameID == '400547827':
            if driveNo == 0 and playNo == 0:
                newtext = "Mitchell Ludwig kickoff , Andrew Motuapuaka 11 Yd Fumble Return (Joey Slye PAT blocked)"     
        if gameID == '400548109':
            if driveNo == 9 and playNo == 0:
                newtext = "Adam Butler 0 Yd Fumble Return for a TD"               
        if gameID == '400547980':
            if driveNo == 3 and playNo == 5:
                newtext = "Austin Collinsworth 32 Yd Fumble Return for a TD (Defensive PAT)"
        if gameID == '401021670':
            if driveNo == 22 and playNo == 3:
                newtext = "Cephus Johnson pass intercepted for a TD Alvin Ward Jr. return for 28 yds for a TD, (Tyler Bass KICK)"
                
        return newtext,keep    
    

    def addPlay(self, gameID, driveNo, prevPlayNo, prevPlay, possData, debug=False):
        debug = True
        text = None
        if gameID == "401012282":
            if driveNo == 8 and prevPlayNo == 11:
                text = "Tyrel Dodson 78 Yd Return of Blocked Field Goal (Seth Small Kick)"
                print(prevPlay)

        if text is not None:
            ptype = playtype(text)
            play  = ptype.getPlay()
            play.analyze(debug=debug)
            playPossession = possData.determinePossession(play, debug=debug)
            playStart = prevPlay.start
            playResult = playclass(possession=playPossession, start=playStart, play=play, valid=True)
            if debug:
                print("Adding play [{0}] for team [{1}] with text [{2}]".format(play.name, playPossession.start, text))
            return playResult
        
        return None
            
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Apr 14, 2019 21:32:15 for Last Run


In [74]:
#tsc = teamstatisticsclass()
#tsc.collect(hist, test=False, debug=True)

In [76]:
pbp = playbyplay()
pbp.setHistorical(hist)
pbp.parseGames(gameID=None, test=False, debug=False, verydebug=False)

/Users/tgadfort/Documents/football/results/games/2009-games.p
/Users/tgadfort/Documents/football/results/games/2010-games.p
/Users/tgadfort/Documents/football/results/games/2011-games.p
/Users/tgadfort/Documents/football/results/games/2008-games.p
/Users/tgadfort/Documents/football/results/games/2018-games.p
GAME ID: 401012246
  Found recovering team: LOU
FUNC splitFumble
	Splitting Fumble: Drive 2, Play 7, and Text Tua Tagovailoa pass complete to DeVonta Smith for 18 yds DeVonta Smith fumbled, forced by Cornelius Sturghill, recovered by Lvile Dorian Etheridge
	Result of a lost fumble is True
	Old possession is ALA
	New possession is LOU
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      7   7   0   0   ['Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds for a TD, (Austin Jones KICK)', 'Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds for a TD, (Austin Jones KICK)']
1      7   7   0   0   []
2      7   7   0   0   []
3      7   7   0   0   []
4      14  

FUNC splitFumble
	Splitting Fumble: Drive 2, Play 8, and Text TEAM run for a loss of 4 yards to the CCaro 8 TEAM fumbled, recovered by SCaro Jake Bentley
	Result of a lost fumble is False
	Old possession is SC
	New possession is SC
	Splitting Fumble: Drive 8, Play 1, and Text Jake Bentley pass complete to Bryan Edwards for 6 yds Bryan Edwards fumbled, forced by C.J. Brewer, , return to the 50 yard line for a 1ST down
	Result of a lost fumble is False
	Old possession is SC
	New possession is SC
	Splitting Fumble: Drive 14, Play 5, and Text Alex James run for a loss of 2 yards to the CCaro 37 Alex James fumbled, recovered by SCaro R Louis Jr.
	Result of a lost fumble is True
	Old possession is CCU
	New possession is SC
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      7   7   0   0   ['Rico Dowdle run for 5 yds for a TD, (Parker White KICK)', 'Rico Dowdle run for 5 yds for a TD, (Parker White KICK)']
1      7   7   0   0   []
2      14  14  0   0   ['Jake Bentley pa

FUNC splitFumble
	Splitting Fumble: Drive 5, Play 9, and Text Terry Wilson run for 2 yds to the Fla 33 Terry Wilson fumbled, forced by James Houston IV, recovered by Fla Jeremiah Moon
	Result of a lost fumble is True
	Old possession is UK
	New possession is FLA
	Splitting Fumble: Drive 16, Play 5, and Text Feleipe Franks run for a loss of 1 yard to the Fla 47 Feleipe Franks fumbled, recovered by Fla Feleipe Franks
	Result of a lost fumble is False
	Old possession is FLA
	New possession is FLA
	Splitting Fumble: Drive 18, Play 2, and Text Feleipe Franks sacked by Josh Allen for 5 yards Feleipe Franks fumbled, recovered by Kent Davonte Robinson , return for 30 yds
	Result of a lost fumble is True
	Old possession is FLA
	New possession is UK
  Last Possession goes to UK
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      0   0   7   7   ['Terry Wilson pass complete to David Bouvier for 29 yds for a TD, (Miles Butler KICK)', 'Terry Wilson pass 

FUNC splitFumble
	Splitting Fumble: Drive 0, Play 9, and Text Kyle Shurmur pass complete to Chris Pierce for 5 yds Chris Pierce fumbled, forced by Hausia Sekona, recovered by Nevad Dom Peterson
	Result of a lost fumble is True
	Old possession is VAN
	New possession is NEV
	Splitting Fumble: Drive 5, Play 3, and Text Ty Gangi sacked by Kenny Hebert for a loss of 8 yards to the Nevad 7 Ty Gangi fumbled, recovered by Vandy Kenny Hebert , return for 0 yards
	Result of a lost fumble is True
	Old possession is NEV
	New possession is VAN
	Splitting Fumble: Drive 8, Play 4, and Text Kyle Shurmur pass complete to Kalija Lipscomb for 5 yds Kalija Lipscomb fumbled, forced by Malik Reed, recovered by Nevad Justin Brent
	Result of a lost fumble is True
	Old possession is VAN
	New possession is NEV
	Splitting Fumble: Drive 19, Play 6, and Text Ty Gangi sacked for a loss of 7 yards to the Nevad 48 Ty Gangi fumbled, recovered by Nevad Toa Taua , return for 0 yards
	Result of a lost fumble is False
	Ol

FUNC splitFumble
	Splitting Fumble: Drive 1, Play 4, and Text Kai Locksley pass complete to Forest McKee for 2 yds Forest McKee fumbled, forced by Bryce Thompson, , return to the 50 yard line
	Result of a lost fumble is False
	Old possession is UTEP
	New possession is UTEP
	Splitting Fumble: Drive 7, Play 6, and Text Mitchell Crawford punt for 45 yds , Marquez Callaway returns for a loss of 2 yards to the Tenn 33 Marquez Callaway fumbled, recovered by UTEP TK Powell
	Result of a lost fumble is True
	Old possession is TENN
	New possession is UTEP
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      0   0   0   0   []
2      3   3   0   0   ['Brent Cimaglia 38 yd FG GOOD']
3      3   3   0   0   []
4      3   3   0   0   []
5      3   3   0   0   []
6      10  10  0   0   ['Tim Jordan run for 3 yds for a TD, (Brent Cimaglia KICK)', 'Tim Jordan run for 3 yds for a TD, (Brent Cimaglia KICK)']
7      10  10  0   0   []
8      10  10  0   0   []
9

...

FUNC splitFumble
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      0   0   0   0   []
2      0   0   0   0   []
3      0   0   3   3   ['Evan Staley 28 yd FG GOOD']
4      7   7   3   3   ['Abdul Adams run for 4 yds for a TD, (Andre Szmyt KICK)', 'Abdul Adams run for 4 yds for a TD, (Andre Szmyt KICK)']
5      7   7   3   3   []
6      7   7   3   3   []
7      7   7   9   9   ['Kennedy McKoy run for 3 yds for a TD, (Evan Staley PAT MISSED)']
8      7   7   9   9   []
9      7   7   9   9   []
10     14  14  9   9   ['Abdul Adams run for 1 yd for a TD, (Andre Szmyt KICK)', 'Abdul Adams run for 1 yd for a TD, (Andre Szmyt KICK)']
11     14  14  12  12  ['Evan Staley 36 yd FG GOOD']
12     14  14  12  12  []
13     14  14  12  12  []
14     14  14  12  12  []
15     14  14  12  12  []
16     14  14  12  12  []
17     14  14  12  12  []
18     14  14  15  15  ['Evan Staley 44 yd FG GOOD']
19     17  17  15  15  ['Andre Szmyt 39 yd FG GOOD']


	Splitting Fumble: Drive 9, Play 6, and Text Matt Hockett punt for 54 yds , Richaud Floyd returns for a loss of 7 yards to the Misso 13 Richaud Floyd fumbled, recovered by Misso Jarvis Ware
	Result of a lost fumble is True
	Old possession is MIZ
	New possession is OKST
	Splitting Fumble: Drive 13, Play 2, and Text Drew Lock pass complete to Jalen Knox for 6 yds Jalen Knox fumbled, forced by Kenneth Edison-McGruder, recovered by OKSt Justin Phillips
	Result of a lost fumble is True
	Old possession is MIZ
	New possession is OKST
	Splitting Fumble: Drive 17, Play 2, and Text Tyler Badie run for a loss of 3 yards to the Misso 22 Tyler Badie fumbled, forced by Jarrell Owens, recovered by Misso
	Result of a lost fumble is True
	Old possession is MIZ
	New possession is OKST
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      0   0   0   0   []
2      7   7   3   3   ['Dillon Stoner 30 Yd pass from Taylor Cornelius (Matt Ammendola Kick)', 'Dillon S

  Found recovering team: STAN
FUNC splitFumble
	Splitting Fumble: Drive 15, Play 3, and Text K.J. Costello sacked by Patrick Jones II for a loss of 11 yards to the Stanf 15 K.J. Costello fumbled, forced by Patrick Jones II, recovered by Stanf
	Result of a lost fumble is False
	Old possession is STAN
	New possession is STAN
	Splitting Fumble: Drive 19, Play 9, and Text K.J. Costello run for 1 yd K.J. Costello fumbled, forced by Dewayne Hendrix, recovered by Stanf Cameron Scarlett for a TD, (Jet Toner KICK)
	Using previous fumble data to set possession to STAN
	Old possession is PITT
	New possession is STAN
	Splitting Fumble: Drive 20, Play 9, and Text Kenny Pickett run for a loss of 2 yards to the Stanf 37 Kenny Pickett fumbled, recovered by Pitt Kenny Pickett
	Result of a lost fumble is False
	Old possession is PITT
	New possession is PITT
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   0   0   []
1      0   0   0   0   []
2      0   0   0   0   []
3    

FUNC splitFumble
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      0   0   3   3   ['Kyle Pfau 37 yd FG GOOD']
1      7   7   3   3   ['Marcus Williams Jr. run for 1 yd for a TD, (Chandler Staton KICK)', 'Marcus Williams Jr. run for 1 yd for a TD, (Chandler Staton KICK)']
2      7   7   3   3   []
3      7   7   3   3   []
4      7   7   3   3   []
5      7   7   3   3   []
6      7   7   3   3   []
7      7   7   3   3   []
8      7   7   6   6   ['Kyle Pfau 45 yd FG GOOD']
9      14  14  6   6   ['Zac Thomas run for 25 yds for a TD, (Chandler Staton KICK)', 'Zac Thomas run for 25 yds for a TD, (Chandler Staton KICK)']
10     14  14  6   6   []
11     14  14  6   6   []
12     14  14  6   6   []
13     17  17  6   6   ['Chandler Staton 42 yd FG GOOD']
14     17  17  13  13  ['Andre Nunez run for 8 yds for a TD, (Kyle Pfau KICK)', 'Andre Nunez run for 8 yds for a TD, (Kyle Pfau KICK)']
15     17  17  13  13  []
16     17  17  16  16  ['Kyle Pfau 23 yd FG GOOD']
17

	Old possession is NCST
	New possession is ECU
	Splitting Fumble: Drive 28, Play 3, and Text Caiden Norman run for a loss of 3 yards to the NCSt 29 Caiden Norman fumbled, recovered by ECaro Anthony Scott
	Result of a lost fumble is True
	Old possession is ECU
	New possession is NCST
FUNC analyzeGameScore
Drive  H   (H) A   (A) 	(Post Drive Score)
0      3   3   0   0   ['Christopher Dunn 23 yd FG GOOD']
1      3   3   0   0   []
2      10  10  0   0   ['Ryan Finley pass complete to Kelvin Harmon for 14 yds for a TD, (Christopher Dunn KICK)', 'Ryan Finley pass complete to Kelvin Harmon for 14 yds for a TD, (Christopher Dunn KICK)']
3      10  10  0   0   []
4      17  17  0   0   ['Reggie Gallaspy II run for 11 yds for a TD, (Christopher Dunn KICK)', 'Reggie Gallaspy II run for 11 yds for a TD, (Christopher Dunn KICK)']
5      17  17  0   0   []
6      24  24  0   0   ['Ryan Finley pass complete to Jakobi Meyers for 24 yds for a TD, (Christopher Dunn KICK)', 'Ryan Finley pass complete t

In [77]:
print(len(pbp.badGames.keys())+len(pbp.goodGames.keys()))
print(len(pbp.badGames.keys()))
print(pbp.badGames.keys())

769
30
dict_keys(['401012300', '401012307', '401012341', '401012342', '401012347', '401012726', '401012823', '401013043', '401013134', '401013155', '401013156', '401013163', '401013177', '401013354', '401013365', '401013372', '401013444', '401015024', '401015032', '401019511', '401020753', '401020758', '401020761', '401021669', '401021683', '401021685', '401021698', '401022547', '401022548', '401035252'])


In [62]:
def writeEdit(gameID, driveNo, playNo, text=None):
    print("        if gameID == '{0}':".format(gameID))
    print("            if driveNo == {0} and playNo == {1}:".format(driveNo, playNo))
    if text is not None:
        print("                newtext = \"{0}\"".format(text))
    else:
        print("                keep = False")

        

writeEdit(401021670, 22, 3, "Cephus Johnson pass intercepted for a TD Alvin Ward Jr. return for 28 yds for a TD, (Tyler Bass KICK)")

        if gameID == '401021670':
            if driveNo == 22 and playNo == 3:
                newtext = "Cephus Johnson pass intercepted for a TD Alvin Ward Jr. return for 28 yds for a TD, (Tyler Bass KICK)"
